# Import + CSV

In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

# EDA

### 0. 결측치가 많은 feature 7개 drop

In [5]:
train.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'], inplace=True)
test.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'], inplace=True)

train.drop(columns=['착상 전 유전 검사 사용 여부'], inplace=True)
test.drop(columns=['착상 전 유전 검사 사용 여부'], inplace=True)

train.drop(columns=['난자 해동 경과일'], inplace=True)
test.drop(columns=['난자 해동 경과일'], inplace=True)

train.drop(columns=['배아 해동 경과일'], inplace=True)
test.drop(columns=['배아 해동 경과일'], inplace=True)

train.drop(columns=['PGS 시술 여부'], inplace=True)
test.drop(columns=['PGS 시술 여부'], inplace=True)

train.drop(columns=['PGD 시술 여부'], inplace=True)
test.drop(columns=['PGD 시술 여부'], inplace=True)

train.drop(columns=['난자 채취 경과일'], inplace=True)
test.drop(columns=['난자 채취 경과일'], inplace=True)

### 1. 난자 혼합 경과일, 배아 이식 경과일 결측치 채우기 (검토 필요)

In [7]:
a = '배아 이식 경과일'
b = '난자 혼합 경과일'

# 배아 이식 경과일 → 최빈값 처리
train[a] = train[a].fillna(train[a].mode().iloc[0])
test[a] = test[a].fillna(test[a].mode().iloc[0])

# 난자 혼합 경과일 → 중앙값 처리
train[b] = train[b].fillna(train[b].median())
test[b] = test[b].fillna(test[b].median())

# 일단은 최빈값-중앙값 조합이 가장 좋다

난자 혼합 경과일
0.0    201920
1.0       488
2.0       102
3.0        68
5.0        24
6.0         8
4.0         5
7.0         1
Name: count, dtype: int64

배아 이식 경과일
5.0    81459
3.0    57924
2.0    35078
0.0    24904
1.0     6053
4.0     4504
6.0     2773
7.0       90
Name: count, dtype: int64

둘이 분포가 다르다..! 처리 방식을 달리 해야할 듯.. 배아 이식 경과일은 평균값, 혼합 경과일은 중앙값?

### 2. 주/부 불임 원인 개수 feature 추가 (검토 필요)

In [10]:
train["주 불임 원인 개수"] = (
    train["남성 주 불임 원인"] + train["여성 주 불임 원인"] + train["부부 주 불임 원인"])
test["주 불임 원인 개수"] = (
    test["남성 주 불임 원인"] + test["여성 주 불임 원인"] + test["부부 주 불임 원인"])

train["부 불임 원인 개수"] = (
    train["남성 부 불임 원인"] + train["여성 부 불임 원인"] + train["부부 부 불임 원인"])
test["부 불임 원인 개수"] = (
    test["남성 부 불임 원인"] + test["여성 부 불임 원인"] + test["부부 부 불임 원인"])

# 굳이 필요한가..? 검토 결과로는 필요해보임

### 3. 여성 요인 개수로 정리 (남성 요인은 검토 필요)

In [12]:
train["불임 원인 - 여성 요인"] = (
    train["불임 원인 - 자궁내막증"] + train["불임 원인 - 자궁경부 문제"] 
    + train["불임 원인 - 난관 질환"] + train["불임 원인 - 배란 장애"]
)
test["불임 원인 - 여성 요인"] = (
    test["불임 원인 - 자궁내막증"] + test["불임 원인 - 자궁경부 문제"] 
    + train["불임 원인 - 난관 질환"] + test["불임 원인 - 배란 장애"]
)

In [13]:
"""
train["불임 원인 - 남성 요인 개수"] = (
    train["불임 원인 - 정자 형태"] + 
    train["불임 원인 - 정자 운동성"] + 
    train["불임 원인 - 정자 면역학적 요인"] + 
    train["불임 원인 - 정자 농도"]
)

test["불임 원인 - 남성 요인 개수"] = (
    test["불임 원인 - 정자 형태"] + 
    test["불임 원인 - 정자 운동성"] + 
    test["불임 원인 - 정자 면역학적 요인"] + 
    test["불임 원인 - 정자 농도"]
)

# 굳이 필요한가..?
"""

# 해보니까 빼는 게 낫다

'\ntrain["불임 원인 - 남성 요인 개수"] = (\n    train["불임 원인 - 정자 형태"] + \n    train["불임 원인 - 정자 운동성"] + \n    train["불임 원인 - 정자 면역학적 요인"] + \n    train["불임 원인 - 정자 농도"]\n)\n\ntest["불임 원인 - 남성 요인 개수"] = (\n    test["불임 원인 - 정자 형태"] + \n    test["불임 원인 - 정자 운동성"] + \n    test["불임 원인 - 정자 면역학적 요인"] + \n    test["불임 원인 - 정자 농도"]\n)\n\n# 굳이 필요한가..?\n'

### 4. 배아/난자 수 결측치 -999 처리 (+범주화? 검토 필요) ********************************

In [15]:
# 각 특성에 대해 구간 정의 및 라벨링
embryo_bins = [0, 1, 3, 6, 11, 16, 21, float('inf')]
embryo_labels = ['0개 (배아 동결 없음)', '1~2개 (소량 저장)', '3~5개 (보편적인 저장 수준)', 
                 '6~10개 (여러 번 이식 가능)', '11~15개 (배아가 많이 저장됨)', 
                 '16~20개 (고반응군, 다배아 보유)', '21개 이상 (극단적인 고반응)']

egg_bins = [0, 1, 4, 7, 11, 16, 21, float('inf')]
egg_labels = ['0개 (난자 채취 실패)', '1~3개 (난소 저반응, 매우 적음)', '4~6개 (난소 저반응, 적음)', 
              '7~10개 (보편적인 난자 채취 수)', '11~15개 (난소 고반응 가능성)', 
              '16~20개 (고반응, 과배란 가능성)', '21개 이상 (극단적인 난소 과반응)']

embryo_creation_bins = [0, 1, 3, 6, 11, 16, 21, float('inf')]
embryo_creation_labels = ['0개 (배아 생성 실패)', '1~2개 (저반응군)', '3~5개 (중간 반응군)', 
                          '6~10개 (적절한 배아 생성)', '11~15개 (고반응군)', 
                          '16~20개 (매우 높은 반응군)', '21개 이상 (과자극군)']

total_embryo_bins = [0, 1, 3, 6, 11, 21, float('inf')]
total_embryo_labels = ['0개 (배아 없음)', '1~2개 (저반응군)', '3~5개 (중간 반응군)', 
                       '6~10개 (고반응군)', '11~20개 (과반응 가능성)', 
                       '21개 이상 (과자극군)']

stored_egg_bins = [0, 1, 4, 7, 11, float('inf')]
stored_egg_labels = ['0개 (배아 저장 없음)', '1~3개 (매우 적음)', '4~6개 (적음)', 
                     '7~10개 (보편적인 저장량)', '11개 이상 (다량 저장)']

defrosted_embryo_bins = [0, 1, 2, 4, 7, 11, float('inf')]
defrosted_embryo_labels = ['0개 (배아 해동 안 함)', '1개 (단일 배아 해동)', '2~3개 (최소 해동)', 
                           '4~6개 (중간 해동)', '7~10개 (다수 해동)', '11개 이상 (대량 해동)']

defrosted_egg_bins = [0, 1, 4, 8, 13, 21, float('inf')]
defrosted_egg_labels = ['0개 (해동하지 않음)', '1~3개 (극소수 해동)', '4~7개 (소량 해동)', 
                        '8~12개 (일반적인 해동 범위)', '13~20개 (다수 해동)', '21개 이상 (대량 해동)']

# 각 구간을 매핑하는 방법 정의
def map_category(data, bins, labels):
    return pd.cut(data, bins=bins, labels=labels, right=False)

# 각 특성에 대해 구간화 및 매핑
train['저장된 배아 구간'] = map_category(train['저장된 배아 수'], embryo_bins, embryo_labels)
test['저장된 배아 구간'] = map_category(test['저장된 배아 수'], embryo_bins, embryo_labels)

train['난자 채취 구간'] = map_category(train['수집된 신선 난자 수'], egg_bins, egg_labels)
test['난자 채취 구간'] = map_category(test['수집된 신선 난자 수'], egg_bins, egg_labels)

train['배아 생성 구간'] = map_category(train['미세주입에서 생성된 배아 수'], embryo_creation_bins, embryo_creation_labels)
test['배아 생성 구간'] = map_category(test['미세주입에서 생성된 배아 수'], embryo_creation_bins, embryo_creation_labels)

train['총 생성 배아 구간'] = map_category(train['총 생성 배아 수'], total_embryo_bins, total_embryo_labels)
test['총 생성 배아 구간'] = map_category(test['총 생성 배아 수'], total_embryo_bins, total_embryo_labels)

train['저장된 신선 난자 구간'] = map_category(train['저장된 신선 난자 수'], stored_egg_bins, stored_egg_labels)
test['저장된 신선 난자 구간'] = map_category(test['저장된 신선 난자 수'], stored_egg_bins, stored_egg_labels)

train['해동된 배아 구간'] = map_category(train['해동된 배아 수'], defrosted_embryo_bins, defrosted_embryo_labels)
test['해동된 배아 구간'] = map_category(test['해동된 배아 수'], defrosted_embryo_bins, defrosted_embryo_labels)

train['해동 난자 구간'] = map_category(train['해동 난자 수'], defrosted_egg_bins, defrosted_egg_labels)
test['해동 난자 구간'] = map_category(test['해동 난자 수'], defrosted_egg_bins, defrosted_egg_labels)

# 각 구간에 대한 수동 라벨 인코딩
embryo_mapping = {
    '0개 (배아 동결 없음)': 0,
    '1~2개 (소량 저장)': 1,
    '3~5개 (보편적인 저장 수준)': 2,
    '6~10개 (여러 번 이식 가능)': 3,
    '11~15개 (배아가 많이 저장됨)': 4,
    '16~20개 (고반응군, 다배아 보유)': 5,
    '21개 이상 (극단적인 고반응)': 6
}

egg_mapping = {
    '0개 (난자 채취 실패)': 0,
    '1~3개 (난소 저반응, 매우 적음)': 1,
    '4~6개 (난소 저반응, 적음)': 2,
    '7~10개 (보편적인 난자 채취 수)': 3,
    '11~15개 (난소 고반응 가능성)': 4,
    '16~20개 (고반응, 과배란 가능성)': 5,
    '21개 이상 (극단적인 난소 과반응)': 6
}

embryo_creation_mapping = {
    '0개 (배아 생성 실패)': 0,
    '1~2개 (저반응군)': 1,
    '3~5개 (중간 반응군)': 2,
    '6~10개 (적절한 배아 생성)': 3,
    '11~15개 (고반응군)': 4,
    '16~20개 (매우 높은 반응군)': 5,
    '21개 이상 (과자극군)': 6
}

total_embryo_mapping = {
    '0개 (배아 없음)': 0,
    '1~2개 (저반응군)': 1,
    '3~5개 (중간 반응군)': 2,
    '6~10개 (고반응군)': 3,
    '11~20개 (과반응 가능성)': 4,
    '21개 이상 (과자극군)': 5
}

stored_egg_mapping = {
    '0개 (배아 저장 없음)': 0,
    '1~3개 (매우 적음)': 1,
    '4~6개 (적음)': 2,
    '7~10개 (보편적인 저장량)': 3,
    '11개 이상 (다량 저장)': 4
}

defrosted_embryo_mapping = {
    '0개 (배아 해동 안 함)': 0,
    '1개 (단일 배아 해동)': 1,
    '2~3개 (최소 해동)': 2,
    '4~6개 (중간 해동)': 3,
    '7~10개 (다수 해동)': 4,
    '11개 이상 (대량 해동)': 5
}

defrosted_egg_mapping = {
    '0개 (해동하지 않음)': 0,
    '1~3개 (극소수 해동)': 1,
    '4~7개 (소량 해동)': 2,
    '8~12개 (일반적인 해동 범위)': 3,
    '13~20개 (다수 해동)': 4,
    '21개 이상 (대량 해동)': 5
}

# 수동 라벨 인코딩을 적용하는 함수
def apply_label_encoding(df, column_name, mapping, additional_category=None):
    # 'Categorical' 열에 추가 범주를 허용하도록 설정
    if additional_category is not None:
        df[column_name] = df[column_name].cat.add_categories([additional_category])
    
    # 매핑 적용 후 결측치는 -1로 채우고, 수치형으로 변환
    df[column_name] = df[column_name].map(mapping).fillna(-1).astype(float)

# 수동 라벨 인코딩을 각 특성에 대해 적용
apply_label_encoding(train, '저장된 배아 구간', embryo_mapping, additional_category=-1)
apply_label_encoding(test, '저장된 배아 구간', embryo_mapping, additional_category=-1)

apply_label_encoding(train, '난자 채취 구간', egg_mapping, additional_category=-1)
apply_label_encoding(test, '난자 채취 구간', egg_mapping, additional_category=-1)

apply_label_encoding(train, '배아 생성 구간', embryo_creation_mapping, additional_category=-1)
apply_label_encoding(test, '배아 생성 구간', embryo_creation_mapping, additional_category=-1)

apply_label_encoding(train, '총 생성 배아 구간', total_embryo_mapping, additional_category=-1)
apply_label_encoding(test, '총 생성 배아 구간', total_embryo_mapping, additional_category=-1)

apply_label_encoding(train, '저장된 신선 난자 구간', stored_egg_mapping, additional_category=-1)
apply_label_encoding(test, '저장된 신선 난자 구간', stored_egg_mapping, additional_category=-1)

apply_label_encoding(train, '해동된 배아 구간', defrosted_embryo_mapping, additional_category=-1)
apply_label_encoding(test, '해동된 배아 구간', defrosted_embryo_mapping, additional_category=-1)

apply_label_encoding(train, '해동 난자 구간', defrosted_egg_mapping, additional_category=-1)
apply_label_encoding(test, '해동 난자 구간', defrosted_egg_mapping, additional_category=-1)

# 구간화된 열을 원본 열에 넣고 구간 열 삭제하기
train['저장된 배아 수'] = train['저장된 배아 구간']
test['저장된 배아 수'] = test['저장된 배아 구간']

train['수집된 신선 난자 수'] = train['난자 채취 구간']
test['수집된 신선 난자 수'] = test['난자 채취 구간']

train['미세주입에서 생성된 배아 수'] = train['배아 생성 구간']
test['미세주입에서 생성된 배아 수'] = test['배아 생성 구간']

train['총 생성 배아 수'] = train['총 생성 배아 구간']
test['총 생성 배아 수'] = test['총 생성 배아 구간']

train['저장된 신선 난자 수'] = train['저장된 신선 난자 구간']
test['저장된 신선 난자 수'] = test['저장된 신선 난자 구간']

train['해동된 배아 수'] = train['해동된 배아 구간']
test['해동된 배아 수'] = test['해동된 배아 구간']

train['해동 난자 수'] = train['해동 난자 구간']
test['해동 난자 수'] = test['해동 난자 구간']

# 구간 열 삭제하기
train.drop(['저장된 배아 구간', '난자 채취 구간', '배아 생성 구간', '총 생성 배아 구간', 
            '저장된 신선 난자 구간', '해동된 배아 구간', '해동 난자 구간'], axis=1, inplace=True)

test.drop(['저장된 배아 구간', '난자 채취 구간', '배아 생성 구간', '총 생성 배아 구간', 
           '저장된 신선 난자 구간', '해동된 배아 구간', '해동 난자 구간'], axis=1, inplace=True)

# 새로운 특성에 대한 구간 정의 및 라벨링
mixed_egg_bins = [0, 1, 4, 8, 13, 21, float('inf')]
mixed_egg_labels = ['0개 (배아 생성 없음)', '1~3개 (소량 사용)', '4~7개 (일반적인 배아 생성 수)', 
                    '8~12개 (다수 배아 확보 가능)', '13~20개 (대량 사용)', '21개 이상 (과다 사용)']

partner_sperm_bins = [0, 1, 4, 8, 13, 21, float('inf')]
partner_sperm_labels = ['0개 (배아 생성 없음)', '1~3개 (소량 사용)', '4~7개 (일반적인 배아 생성 수)', 
                        '8~12개 (다수 배아 확보 가능)', '13~20개 (대량 사용)', '21개 이상 (과다 사용)']

donor_sperm_bins = [0, 1, 4, 8, 13, 21, float('inf')]
donor_sperm_labels = ['0개 (기증자 정자 사용 안 함)', '1~3개 (소량 사용)', '4~7개 (일반적인 배아 생성 시 기증자 정자 사용 범위)', 
                      '8~12개 (다수 사용)', '13~20개 (대량 사용)', '21개 이상 (과다 사용)']

injected_egg_bins = [0, 1, 4, 7, 11, 16, 21, float('inf')]
injected_egg_labels = ['0개 (배아 생성 실패)', '1~3개 (배아 생성이 매우 적음)', '4~6개 (적정 수준의 배아 생성)', 
                       '7~10개 (충분한 배아 생성)', '11~15개 (다량 배아 생성)', '16~20개 (고반응군)', 
                       '21개 이상 (극단적인 고반응)']

stored_embryo_bins = [0, 1, 2, 3, 6, 11, 21, float('inf')]
stored_embryo_labels = ['0개 (배아 없음)', '1~2개 (저반응군)', '3~5개 (중간 반응군)', '6~10개 (고반응군)', 
                        '11~15개 (다배아 보유 가능성)', '16~20개 (고반응군)', '21개 이상 (과자극군)']

# 구간을 매핑하는 방법 정의
def map_category(data, bins, labels):
    return pd.cut(data, bins=bins, labels=labels, right=False)

# 새로운 특성에 대해 구간화 및 매핑
train['혼합된 난자 수 구간'] = map_category(train['혼합된 난자 수'], mixed_egg_bins, mixed_egg_labels)
test['혼합된 난자 수 구간'] = map_category(test['혼합된 난자 수'], mixed_egg_bins, mixed_egg_labels)

train['파트너 정자와 혼합된 난자 수 구간'] = map_category(train['파트너 정자와 혼합된 난자 수'], partner_sperm_bins, partner_sperm_labels)
test['파트너 정자와 혼합된 난자 수 구간'] = map_category(test['파트너 정자와 혼합된 난자 수'], partner_sperm_bins, partner_sperm_labels)

train['기증자 정자와 혼합된 난자 수 구간'] = map_category(train['기증자 정자와 혼합된 난자 수'], donor_sperm_bins, donor_sperm_labels)
test['기증자 정자와 혼합된 난자 수 구간'] = map_category(test['기증자 정자와 혼합된 난자 수'], donor_sperm_bins, donor_sperm_labels)

train['미세주입된 난자 수 구간'] = map_category(train['미세주입된 난자 수'], injected_egg_bins, injected_egg_labels)
test['미세주입된 난자 수 구간'] = map_category(test['미세주입된 난자 수'], injected_egg_bins, injected_egg_labels)

train['미세주입 후 저장된 배아 수 구간'] = map_category(train['미세주입 후 저장된 배아 수'], stored_embryo_bins, stored_embryo_labels)
test['미세주입 후 저장된 배아 수 구간'] = map_category(test['미세주입 후 저장된 배아 수'], stored_embryo_bins, stored_embryo_labels)

# 각 구간에 대한 수동 라벨 인코딩
mixed_egg_mapping = {
    '0개 (배아 생성 없음)': 0,
    '1~3개 (소량 사용)': 1,
    '4~7개 (일반적인 배아 생성 수)': 2,
    '8~12개 (다수 배아 확보 가능)': 3,
    '13~20개 (대량 사용)': 4,
    '21개 이상 (과다 사용)': 5
}

partner_sperm_mapping = {
    '0개 (배아 생성 없음)': 0,
    '1~3개 (소량 사용)': 1,
    '4~7개 (일반적인 배아 생성 수)': 2,
    '8~12개 (다수 배아 확보 가능)': 3,
    '13~20개 (대량 사용)': 4,
    '21개 이상 (과다 사용)': 5
}

donor_sperm_mapping = {
    '0개 (기증자 정자 사용 안 함)': 0,
    '1~3개 (소량 사용)': 1,
    '4~7개 (일반적인 배아 생성 시 기증자 정자 사용 범위)': 2,
    '8~12개 (다수 사용)': 3,
    '13~20개 (대량 사용)': 4,
    '21개 이상 (과다 사용)': 5
}

injected_egg_mapping = {
    '0개 (배아 생성 실패)': 0,
    '1~3개 (배아 생성이 매우 적음)': 1,
    '4~6개 (적정 수준의 배아 생성)': 2,
    '7~10개 (충분한 배아 생성)': 3,
    '11~15개 (다량 배아 생성)': 4,
    '16~20개 (고반응군)': 5,
    '21개 이상 (극단적인 고반응)': 6
}

stored_embryo_mapping = {
    '0개 (배아 없음)': 0,
    '1~2개 (저반응군)': 1,
    '3~5개 (중간 반응군)': 2,
    '6~10개 (고반응군)': 3,
    '11~15개 (다배아 보유 가능성)': 4,
    '16~20개 (고반응군)': 5,
    '21개 이상 (과자극군)': 6
}

# 수동 라벨 인코딩을 적용하는 함수
def apply_label_encoding(df, column_name, mapping, additional_category=None):
    if additional_category is not None:
        df[column_name] = df[column_name].cat.add_categories([additional_category])
    df[column_name] = df[column_name].map(mapping).fillna(-1).astype(float)

# 수동 라벨 인코딩을 각 특성에 대해 적용
apply_label_encoding(train, '혼합된 난자 수 구간', mixed_egg_mapping, additional_category=-1)
apply_label_encoding(test, '혼합된 난자 수 구간', mixed_egg_mapping, additional_category=-1)

apply_label_encoding(train, '파트너 정자와 혼합된 난자 수 구간', partner_sperm_mapping, additional_category=-1)
apply_label_encoding(test, '파트너 정자와 혼합된 난자 수 구간', partner_sperm_mapping, additional_category=-1)

apply_label_encoding(train, '기증자 정자와 혼합된 난자 수 구간', donor_sperm_mapping, additional_category=-1)
apply_label_encoding(test, '기증자 정자와 혼합된 난자 수 구간', donor_sperm_mapping, additional_category=-1)

apply_label_encoding(train, '미세주입된 난자 수 구간', injected_egg_mapping, additional_category=-1)
apply_label_encoding(test, '미세주입된 난자 수 구간', injected_egg_mapping, additional_category=-1)

apply_label_encoding(train, '미세주입 후 저장된 배아 수 구간', stored_embryo_mapping, additional_category=-1)
apply_label_encoding(test, '미세주입 후 저장된 배아 수 구간', stored_embryo_mapping, additional_category=-1)

# 구간화된 열을 원본 열에 넣고 구간 열 삭제하기
train['혼합된 난자 수'] = train['혼합된 난자 수 구간']
test['혼합된 난자 수'] = test['혼합된 난자 수 구간']

train['파트너 정자와 혼합된 난자 수'] = train['파트너 정자와 혼합된 난자 수 구간']
test['파트너 정자와 혼합된 난자 수'] = test['파트너 정자와 혼합된 난자 수 구간']

train['기증자 정자와 혼합된 난자 수'] = train['기증자 정자와 혼합된 난자 수 구간']
test['기증자 정자와 혼합된 난자 수'] = test['기증자 정자와 혼합된 난자 수 구간']

train['미세주입된 난자 수'] = train['미세주입된 난자 수 구간']
test['미세주입된 난자 수'] = test['미세주입된 난자 수 구간']

train['미세주입 후 저장된 배아 수'] = train['미세주입 후 저장된 배아 수 구간']
test['미세주입 후 저장된 배아 수'] = test['미세주입 후 저장된 배아 수 구간']

# 구간 열 삭제하기
train.drop(['혼합된 난자 수 구간', '파트너 정자와 혼합된 난자 수 구간', '기증자 정자와 혼합된 난자 수 구간', 
            '미세주입된 난자 수 구간', '미세주입 후 저장된 배아 수 구간'], axis=1, inplace=True)

test.drop(['혼합된 난자 수 구간', '파트너 정자와 혼합된 난자 수 구간', '기증자 정자와 혼합된 난자 수 구간', 
           '미세주입된 난자 수 구간', '미세주입 후 저장된 배아 수 구간'], axis=1, inplace=True)

train['이식된 배아 수'] = train['이식된 배아 수'].fillna(-1)
test['이식된 배아 수'] = test['이식된 배아 수'].fillna(-1)

train['미세주입 배아 이식 수'] = train['미세주입 배아 이식 수'].fillna(-1)
test['미세주입 배아 이식 수'] = test['미세주입 배아 이식 수'].fillna(-1)

### 5. 배란 유도 정상화

In [17]:
train["배란 유도 유형"].replace(["세트로타이드 (억제제)", "생식선 자극 호르몬"], "알 수 없음", inplace=True)
test["배란 유도 유형"].replace(["세트로타이드 (억제제)", "생식선 자극 호르몬"], "알 수 없음", inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_23280\1468139943.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["배란 유도 유형"].replace(["세트로타이드 (억제제)", "생식선 자극 호르몬"], "알 수 없음", inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_23280\1468139943.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

### 6. 배아 생성 주요 이유 원-핫 인코딩

In [19]:
# 1. '연구용' 포함 행 삭제 (결측치가 있으면 빈 문자열이므로 함께 처리)
train = train[~train['배아 생성 주요 이유'].fillna('').str.contains('연구용')]
test = test[~test['배아 생성 주요 이유'].fillna('').str.contains('연구용')]

# 결측치 문자열 'Nan'으로 처리
train['배아 생성 주요 이유'] = train['배아 생성 주요 이유'].fillna('Nan')
test['배아 생성 주요 이유'] = test['배아 생성 주요 이유'].fillna('Nan')

# 3. One-hot encoding: 지정된 4가지 항목에 대해 dummy 변수 생성
reasons = ['Nan','기증용', '난자 저장용', '배아 저장용', '현재 시술용']

for reason in reasons:
    col_name = f'reason_{reason}'
    train[col_name] = train['배아 생성 주요 이유'].apply(lambda x: 1 if isinstance(x, str) and reason in x else 0)
    test[col_name] = test['배아 생성 주요 이유'].apply(lambda x: 1 if isinstance(x, str) and reason in x else 0)

# 4. 원래 있던 '배아 생성 주요 이유' 컬럼 삭제
train = train.drop(columns=['배아 생성 주요 이유'])
test = test.drop(columns=['배아 생성 주요 이유'])

### 7. 시술/임신/출산 횟수 정리

In [21]:
valid_values = ['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상']

## train ##
train = train[
    (train['총 시술 횟수'].isin(valid_values)) &
    (train['클리닉 내 총 시술 횟수'].isin(valid_values)) &
    (train['IVF 시술 횟수'].isin(valid_values)) &
    (train['DI 시술 횟수'].isin(valid_values)) &
    (train['총 출산 횟수'].isin(valid_values)) &
    (train['IVF 출산 횟수'].isin(valid_values)) &
    (train['DI 출산 횟수'].isin(valid_values)) &
    (train['총 임신 횟수'].isin(valid_values)) &
    (train['IVF 임신 횟수'].isin(valid_values)) &
    (train['DI 임신 횟수'].isin(valid_values))
]

for col in ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', 
            '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']:
    train[col] = train[col].replace('6회 이상', '6')
    train[col] = train[col].str.replace('회', '', regex=True).astype(int)

train['총 시술 횟수'] = train['IVF 시술 횟수'] + train['DI 시술 횟수']

## test ##
test = test[
    (test['총 시술 횟수'].isin(valid_values)) &
    (test['클리닉 내 총 시술 횟수'].isin(valid_values)) &
    (test['IVF 시술 횟수'].isin(valid_values)) &
    (test['DI 시술 횟수'].isin(valid_values)) &
    (test['총 출산 횟수'].isin(valid_values)) &
    (test['IVF 출산 횟수'].isin(valid_values)) &
    (test['DI 출산 횟수'].isin(valid_values)) &
    (test['총 임신 횟수'].isin(valid_values)) &
    (test['IVF 임신 횟수'].isin(valid_values)) &
    (test['DI 임신 횟수'].isin(valid_values))
]

for col in ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', 
            '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']:
    test[col] = test[col].replace('6회 이상', '6')
    test[col] = test[col].str.replace('회', '', regex=True).astype(int)

test['총 시술 횟수'] = test['IVF 시술 횟수'] + test['DI 시술 횟수']

### 8. 난자/정자 출처 원-핫 인코딩

In [23]:
train = pd.get_dummies(train, columns=["난자 출처", "정자 출처"], dtype=int)
test = pd.get_dummies(test, columns=["난자 출처", "정자 출처"], dtype=int)

### 9. 난자/정자 기증자 나이 라벨 인코딩 (검토 필요)

In [25]:
# 결측치를 -1로 처리 (난자 기증자 나이, 정자 기증자 나이)
train["난자 기증자 나이"].fillna(-1, inplace=True)
test["난자 기증자 나이"].fillna(-1, inplace=True)

train["정자 기증자 나이"].fillna(-1, inplace=True)
test["정자 기증자 나이"].fillna(-1, inplace=True)

# 라벨 인코딩 매핑 딕셔너리
age_mapping_nanja = {
    "만20세 이하": 0,
    "만21-25세": 1,
    "만26-30세": 2,
    "만31-35세": 3,
    "알 수 없음": -1
}

age_mapping_jungja = {
    "알 수 없음": -1, 
    "만21-25세": 1,
    "만26-30세": 2,
    "만31-35세": 3,
    "만36-40세": 4,
    "만41-45세": 5,
    "만20세 이하": 0,
}
    
# 라벨 인코딩 적용(난자)
train["난자 기증자 나이 (라벨 인코딩)"] = train["난자 기증자 나이"].map(age_mapping_nanja)
test["난자 기증자 나이 (라벨 인코딩)"] = test["난자 기증자 나이"].map(age_mapping_nanja)

# 원본 "난자 기증자 나이" 컬럼 제거(난자)
train.drop(columns=["난자 기증자 나이"], inplace=True)
test.drop(columns=["난자 기증자 나이"], inplace=True)

# 라벨 인코딩 적용(정자)
train["정자 기증자 나이 (라벨 인코딩)"] = train["정자 기증자 나이"].map(age_mapping_jungja)
test["정자 기증자 나이 (라벨 인코딩)"] = test["정자 기증자 나이"].map(age_mapping_jungja)

# 원본 "정자 기증자 나이" 컬럼 제거(정자)
train.drop(columns=["정자 기증자 나이"], inplace=True)
test.drop(columns=["정자 기증자 나이"], inplace=True)

# 결측치와 알 수 없음을 같은 -1로 처리? -999로 처리?

C:\Users\USER\AppData\Local\Temp\ipykernel_23280\3223145104.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["난자 기증자 나이"].fillna(-1, inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_23280\3223145104.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

### 10. DI 시술 관련 여부 결측치 -1로 처리 + 원-핫 인코딩

In [27]:
# 결측치를 -1로 채우기
cols_to_fill = ['신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 
                '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '동결 배아 사용 여부']

train[cols_to_fill] = train[cols_to_fill].fillna(-1)
test[cols_to_fill] = test[cols_to_fill].fillna(-1)

# 원핫인코딩 적용
train_encoded = pd.get_dummies(train, columns=cols_to_fill)
test_encoded = pd.get_dummies(test, columns=cols_to_fill)

# 기존 열을 삭제한 데이터프레임
train = train_encoded
test = test_encoded

### 11. 특정 시술 유형 그룹화 및 원-핫 인코딩

In [29]:
# 특정 시술 유형 그룹화 함수
def categorize_treatment(treatment):
    if pd.isna(treatment) or "Unknown" in treatment:
        return "Unknown"
    elif "IVF" in treatment:
        return "IVF 기반"
    elif "ICSI" in treatment:
        return "ICSI 기반"
    elif "Generic DI" in treatment or "IUI" in treatment or "ICI" in treatment or "IVI" in treatment:
        return "DI 기반"
    else:
        return "Unknown"

# 특정 시술 유형 컬럼을 그룹화
train["특정 시술 유형"] = train["특정 시술 유형"].apply(categorize_treatment)
test["특정 시술 유형"] = test["특정 시술 유형"].apply(categorize_treatment)


# One-Hot Encoding 수행
train_encoded = pd.get_dummies(train["특정 시술 유형"], prefix="시술유형")
test_encoded = pd.get_dummies(test["특정 시술 유형"], prefix="시술유형")

# 기존 데이터프레임에 One-Hot Encoding된 컬럼 추가
train = pd.concat([train, train_encoded], axis=1)
test = pd.concat([test, test_encoded], axis=1)

# 원본 "특정 시술 유형" 컬럼 제거 (필요 시)
train.drop(columns=["특정 시술 유형"], inplace=True)
test.drop(columns=["특정 시술 유형"], inplace=True)

### 12. 시술 시기 코드 원-핫 인코딩

In [31]:
# 전처리 함수
def preprocess_timing_code(code):
    return "Unknown" if pd.isna(code) else code

# 결측 처리 적용
train["시술 시기 코드"] = train["시술 시기 코드"].apply(preprocess_timing_code)
test["시술 시기 코드"] = test["시술 시기 코드"].apply(preprocess_timing_code)

# 인코딩 적용 및 컬럼 정렬 (train/test 직접 대체)
train = pd.get_dummies(train, columns=["시술 시기 코드"], prefix="시술시기")
test = pd.get_dummies(test, columns=["시술 시기 코드"], prefix="시술시기")
train, test = train.align(test, join='left', axis=1, fill_value=0)

### 13. 시술 당시 나이 결측치 -999 처리 및 라벨 인코딩 (검토 필요)

In [33]:
# 라벨 인코딩 매핑 딕셔너리 (나이가 작은 순으로 매핑)
age_mapping_treatment = {
    "만18-34세": 0,
    "만35-37세": 1,
    "만38-39세": 2,
    "만40-42세": 3,
    "만43-44세": 4,
    "만45-50세": 5,
    "알 수 없음": -1
}

# 라벨 인코딩 적용
train["시술 당시 나이 (라벨 인코딩)"] = train["시술 당시 나이"].map(age_mapping_treatment)
test["시술 당시 나이 (라벨 인코딩)"] = test["시술 당시 나이"].map(age_mapping_treatment)

# 원본 "시술 당시 나이" 컬럼 제거
train.drop(columns=["시술 당시 나이"], inplace=True)
test.drop(columns=["시술 당시 나이"], inplace=True)

# -1? -999?

### 불명확 불임 원인, 배란 자극 여부, 배란 유도 유형, 시술 유형 라벨 인코딩

In [35]:
label_columns = ['불명확 불임 원인', '배란 자극 여부', '시술 유형', '배란 유도 유형']

In [36]:
for col in label_columns:
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train[label_columns] = ordinal_encoder.fit_transform(train[label_columns])
test[label_columns] = ordinal_encoder.transform(test[label_columns])

### 나이 평균 추가

In [38]:
# 난자 기증자 나이와 시술 당시 나이 컬럼명
donor_age_col = "난자 기증자 나이 (라벨 인코딩)"
procedure_age_col = "시술 당시 나이 (라벨 인코딩)"

def calculate_age_feature(row):
    if row[donor_age_col] == -1 or row[procedure_age_col] == -1:
        return -1  # 결측치가 포함된 경우 -1 유지
    elif row[donor_age_col] < row[procedure_age_col]:
        return row[donor_age_col]  # 난자 기증자가 더 젊다면 그 나이를 사용
    else:
        return np.mean([row[donor_age_col], row[procedure_age_col]])  # 평균값 사용

# 새로운 나이 피처 생성 (train & test)
train["나이 평균"] = train.apply(calculate_age_feature, axis=1)
test["나이 평균"] = test.apply(calculate_age_feature, axis=1)

***

# X와 y로 분리

In [41]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

test.drop(columns=['임신 성공 여부'], inplace=True)

# K-Fold 점수 계산

In [43]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV

# CatBoost 기본 모델 설정
cat_model = CatBoostClassifier(verbose=0, random_state=42)

# 하이퍼파라미터 탐색 공간 정의
param_dist = {
    'iterations': [500, 1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'border_count': [32, 64, 128]
}

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    cat_model,
    param_distributions=param_dist,
    n_iter=20,  # 탐색 횟수 (조정 가능)
    scoring='roc_auc',
    cv=5,
    random_state=42,
    n_jobs=-1
)

# 최적 하이퍼파라미터 찾기
random_search.fit(X, y)

# 최적의 파라미터 출력
print("Best Parameters:", random_search.best_params_)
print("Best ROC-AUC Score:", random_search.best_score_)

KeyboardInterrupt: 

In [50]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Stratified K-Fold 설정
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 최적의 하이퍼파라미터 적용
lgb_model = LGBMClassifier(
    subsample=0.6,
    reg_lambda=10,
    reg_alpha=0,
    num_leaves=31,
    n_estimators=1000,
    min_child_samples=30,
    max_depth=10,
    learning_rate=0.01,
    colsample_bytree=1.0,
    random_state=42
)

xgb_model = XGBClassifier(
    subsample=0.6,
    reg_lambda=0,
    reg_alpha=10,
    n_estimators=1000,
    min_child_weight=3,
    max_depth=3,
    learning_rate=0.05,
    gamma=0.2,
    colsample_bytree=1.0,
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

# CatBoost 모델 설정 (최적 하이퍼파라미터 반영)
cat_model = CatBoostClassifier(
    learning_rate=0.05,
    l2_leaf_reg=1,          # 최적의 l2_leaf_reg 파라미터 적용
    iterations=500,         # 최적의 iterations 값 적용
    depth=6,                # 최적의 depth 값 적용
    border_count=64,        # 최적의 border_count 값 적용
    task_type='CPU',        # GPU 사용 설정
    devices='0',            # 첫 번째 GPU 사용 (필요시 다른 GPU로 변경)
    random_state=42,
    verbose=0
)

# 교차 검증 수행
lgb_auc_scores = []
xgb_auc_scores = []
cat_auc_scores = []

for train_idx, val_idx in kf.split(X, y):
    X_train_part, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train_part, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # LightGBM 학습
    lgb_model.fit(X_train_part, y_train_part)
    lgb_val_pred_proba = lgb_model.predict_proba(X_val)[:, 1]
    lgb_auc_scores.append(roc_auc_score(y_val, lgb_val_pred_proba))

    # XGBoost 학습
    xgb_model.fit(X_train_part, y_train_part)
    xgb_val_pred_proba = xgb_model.predict_proba(X_val)[:, 1]
    xgb_auc_scores.append(roc_auc_score(y_val, xgb_val_pred_proba))

    # CatBoost 학습
    cat_model.fit(X_train_part, y_train_part)
    cat_val_pred_proba = cat_model.predict_proba(X_val)[:, 1]
    cat_auc_scores.append(roc_auc_score(y_val, cat_val_pred_proba))

# 평균 ROC-AUC 점수 출력
mean_lgb_auc = np.mean(lgb_auc_scores)
mean_xgb_auc = np.mean(xgb_auc_scores)
mean_cat_auc = np.mean(cat_auc_scores)

print(f"Mean ROC-AUC Score (10-Fold CV) - LightGBM: {mean_lgb_auc:.10f}")
print(f"Mean ROC-AUC Score (10-Fold CV) - XGBoost: {mean_xgb_auc:.10f}")
print(f"Mean ROC-AUC Score (10-Fold CV) - CatBoost: {mean_cat_auc:.10f}")

# 전체 데이터로 모델 재학습
lgb_model.fit(X, y)
xgb_model.fit(X, y)
cat_model.fit(X, y)

# test 데이터 예측
lgb_test_pred_proba = lgb_model.predict_proba(test)[:, 1]
xgb_test_pred_proba = xgb_model.predict_proba(test)[:, 1]
cat_test_pred_proba = cat_model.predict_proba(test)[:, 1]

C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [06:40:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [06:41:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [06:41:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\xgbo

Mean ROC-AUC Score (10-Fold CV) - LightGBM: 0.7393665866
Mean ROC-AUC Score (10-Fold CV) - XGBoost: 0.7396686170
Mean ROC-AUC Score (10-Fold CV) - CatBoost: 0.7394016921


C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [06:47:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [77]:
ratios = [
    (0.33, 0.33, 0.33),  # 세 모델을 동일 비율로 섞기
    (0.4, 0.4, 0.2),     # LightGBM과 CatBoost를 더 많이 반영
    (0.45, 0.35, 0.2),   # LightGBM을 조금 더 반영
    (0.5, 0.3, 0.2),     # LightGBM 50%, CatBoost 30%, XGBoost 20%
    (0.35, 0.45, 0.2),   # CatBoost를 조금 더 반영
    (0.6, 0.2, 0.2),     # LightGBM 60%, CatBoost와 XGBoost 20%씩
    (0.3, 0.5, 0.2),     # CatBoost를 더 많이 반영
    (0.25, 0.5, 0.25),   # CatBoost 50%, 나머지 두 모델 25%씩
    (0.4, 0.2, 0.4),     # LightGBM과 XGBoost를 비슷하게 반영
    (0.3, 0.4, 0.3),     # LightGBM과 CatBoost를 비슷하게 반영
]

best_auc = 0
best_ratio = None
best_ensemble_valid_proba = None

# 최적의 비율 찾기
for lgb_ratio, xgb_ratio, cat_ratio in ratios:
    ensemble_valid_proba = (lgb_ratio * lgb_val_pred_proba) + (xgb_ratio * xgb_val_pred_proba) + (cat_ratio * cat_val_pred_proba)
    auc = roc_auc_score(y_val, ensemble_valid_proba)
    print(f"LGB: {lgb_ratio}, XGB: {xgb_ratio}, CAT: {cat_ratio}, AUC: {auc:.6f}")
    
    if auc > best_auc:
        best_auc = auc
        best_ratio = (lgb_ratio, xgb_ratio, cat_ratio)
        best_ensemble_valid_proba = ensemble_valid_proba

# 최적 비율 출력
print(f"Best Ratio - LGB: {best_ratio[0]}, XGB: {best_ratio[1]}, CAT: {best_ratio[2]}, Best AUC: {best_auc:.10f}")

# 최적 비율을 사용하여 테스트 데이터 예측 계산
ensemble_test_proba = (best_ratio[0] * lgb_test_pred_proba) + (best_ratio[1] * xgb_test_pred_proba) + (best_ratio[2] * cat_test_pred_proba)

# 예측 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = ensemble_test_proba
sample_submission.to_csv('./02_27_XG_LGB_CAT_최적비율_2.csv', index=False)

LGB: 0.33, XGB: 0.33, CAT: 0.33, AUC: 0.737156
LGB: 0.4, XGB: 0.4, CAT: 0.2, AUC: 0.737119
LGB: 0.45, XGB: 0.35, CAT: 0.2, AUC: 0.737123
LGB: 0.5, XGB: 0.3, CAT: 0.2, AUC: 0.737121
LGB: 0.35, XGB: 0.45, CAT: 0.2, AUC: 0.737110
LGB: 0.6, XGB: 0.2, CAT: 0.2, AUC: 0.737100
LGB: 0.3, XGB: 0.5, CAT: 0.2, AUC: 0.737088
LGB: 0.25, XGB: 0.5, CAT: 0.25, AUC: 0.737092
LGB: 0.4, XGB: 0.2, CAT: 0.4, AUC: 0.737177
LGB: 0.3, XGB: 0.4, CAT: 0.3, AUC: 0.737135
Best Ratio - LGB: 0.4, XGB: 0.2, CAT: 0.4, Best AUC: 0.7371766639


submit: 0.7371766639
best: 0.7371783875

LGB: 0.33, XGB: 0.33, CAT: 0.33, AUC: 0.736971
LGB: 0.4, XGB: 0.4, CAT: 0.2, AUC: 0.737002
LGB: 0.45, XGB: 0.35, CAT: 0.2, AUC: 0.737011
LGB: 0.5, XGB: 0.3, CAT: 0.2, AUC: 0.737009
LGB: 0.35, XGB: 0.45, CAT: 0.2, AUC: 0.736995
LGB: 0.6, XGB: 0.2, CAT: 0.2, AUC: 0.736992
LGB: 0.3, XGB: 0.5, CAT: 0.2, AUC: 0.736977
LGB: 0.25, XGB: 0.5, CAT: 0.25, AUC: 0.736952
LGB: 0.4, XGB: 0.2, CAT: 0.4, AUC: 0.736956
LGB: 0.3, XGB: 0.4, CAT: 0.3, AUC: 0.736968
Best Ratio - LGB: 0.45, XGB: 0.35, CAT: 0.2, Best AUC: 0.737011

cat 튜닝 적당히

LGB: 0.33, XGB: 0.33, CAT: 0.33, AUC: 0.737156
LGB: 0.4, XGB: 0.4, CAT: 0.2, AUC: 0.737119
LGB: 0.45, XGB: 0.35, CAT: 0.2, AUC: 0.737123
LGB: 0.5, XGB: 0.3, CAT: 0.2, AUC: 0.737121
LGB: 0.35, XGB: 0.45, CAT: 0.2, AUC: 0.737110
LGB: 0.6, XGB: 0.2, CAT: 0.2, AUC: 0.737100
LGB: 0.3, XGB: 0.5, CAT: 0.2, AUC: 0.737088
LGB: 0.25, XGB: 0.5, CAT: 0.25, AUC: 0.737092
LGB: 0.4, XGB: 0.2, CAT: 0.4, AUC: 0.737177
LGB: 0.3, XGB: 0.4, CAT: 0.3, AUC: 0.737135


Best Ratio - LGB: 0.4, XGB: 0.2, CAT: 0.4, Best AUC: 0.737177

***

In [59]:
ratios = [
    (0.35, 0.25, 0.4),  # LightGBM을 조금 더 높이고, XGBoost를 낮추고 CatBoost를 높이기
    (0.45, 0.25, 0.3),  # LightGBM을 많이 반영하고 CatBoost와 XGBoost는 비슷하게
    (0.4, 0.3, 0.3),    # LightGBM과 XGBoost를 비슷하게, CatBoost는 좀 더 비중을 높이기
    (0.5, 0.25, 0.25),  # LightGBM을 절반 이상 반영하고, 나머지 두 모델은 적당히
    (0.4, 0.25, 0.35),  # LightGBM을 조금 더 반영하고, CatBoost를 좀 더 비중을 높이기
    (0.45, 0.3, 0.25),  # LightGBM을 많이 반영하고 XGBoost와 CatBoost는 비슷하게
    (0.55, 0.2, 0.25),  # LightGBM을 더 비중을 높이고, XGBoost를 적당히
    (0.3, 0.3, 0.4),    # XGBoost와 CatBoost를 비슷하게, LightGBM을 낮추기
]

best_auc = 0
best_ratio = None
best_ensemble_valid_proba = None

# 최적의 비율 찾기
for lgb_ratio, xgb_ratio, cat_ratio in ratios:
    ensemble_valid_proba = (lgb_ratio * lgb_val_pred_proba) + (xgb_ratio * xgb_val_pred_proba) + (cat_ratio * cat_val_pred_proba)
    auc = roc_auc_score(y_val, ensemble_valid_proba)
    print(f"LGB: {lgb_ratio}, XGB: {xgb_ratio}, CAT: {cat_ratio}, AUC: {auc:.6f}")
    
    if auc > best_auc:
        best_auc = auc
        best_ratio = (lgb_ratio, xgb_ratio, cat_ratio)
        best_ensemble_valid_proba = ensemble_valid_proba

# 최적 비율 출력
print(f"Best Ratio - LGB: {best_ratio[0]}, XGB: {best_ratio[1]}, CAT: {best_ratio[2]}, Best AUC: {best_auc:.6f}")

# 최적 비율을 사용하여 테스트 데이터 예측 계산
ensemble_test_proba = (best_ratio[0] * lgb_test_pred_proba) + (best_ratio[1] * xgb_test_pred_proba) + (best_ratio[2] * cat_test_pred_proba)

# 예측 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = ensemble_test_proba
sample_submission.to_csv('./02_27_XG_LGB_CAT_최적비율_3.csv', index=False)

LGB: 0.35, XGB: 0.25, CAT: 0.4, AUC: 0.737170
LGB: 0.45, XGB: 0.25, CAT: 0.3, AUC: 0.737155
LGB: 0.4, XGB: 0.3, CAT: 0.3, AUC: 0.737153
LGB: 0.5, XGB: 0.25, CAT: 0.25, AUC: 0.737140
LGB: 0.4, XGB: 0.25, CAT: 0.35, AUC: 0.737164
LGB: 0.45, XGB: 0.3, CAT: 0.25, AUC: 0.737144
LGB: 0.55, XGB: 0.2, CAT: 0.25, AUC: 0.737128
LGB: 0.3, XGB: 0.3, CAT: 0.4, AUC: 0.737163
Best Ratio - LGB: 0.35, XGB: 0.25, CAT: 0.4, Best AUC: 0.737170


In [75]:
ratios = [
    (0.30, 0.15, 0.55),  # LGB 30%, XGBoost 15%, CAT 55%
    (0.31, 0.15, 0.54),  # LGB 31%, XGBoost 15%, CAT 54%
    (0.32, 0.15, 0.53),  # LGB 32%, XGBoost 15%, CAT 53%
    (0.33, 0.15, 0.52),  # LGB 33%, XGBoost 15%, CAT 52%
    (0.30, 0.16, 0.54),  # LGB 30%, XGBoost 16%, CAT 54%
    (0.31, 0.16, 0.53),  # LGB 31%, XGBoost 16%, CAT 53%
    (0.32, 0.16, 0.52),  # LGB 32%, XGBoost 16%, CAT 52%
    (0.33, 0.16, 0.51),  # LGB 33%, XGBoost 16%, CAT 51%
    (0.30, 0.17, 0.53),  # LGB 30%, XGBoost 17%, CAT 53%
    (0.31, 0.17, 0.52),  # LGB 31%, XGBoost 17%, CAT 52%
    (0.32, 0.17, 0.51),  # LGB 32%, XGBoost 17%, CAT 51%
    (0.33, 0.17, 0.50),  # LGB 33%, XGBoost 17%, CAT 50%
    (0.30, 0.18, 0.52),  # LGB 30%, XGBoost 18%, CAT 52%
    (0.31, 0.18, 0.51),  # LGB 31%, XGBoost 18%, CAT 51%
    (0.32, 0.18, 0.50),  # LGB 32%, XGBoost 18%, CAT 50%
    (0.33, 0.18, 0.49),  # LGB 33%, XGBoost 18%, CAT 49%
    (0.29, 0.15, 0.56),  # LGB 29%, XGBoost 15%, CAT 56%
    (0.30, 0.14, 0.56),  # LGB 30%, XGBoost 14%, CAT 56%
    (0.31, 0.14, 0.55),  # LGB 31%, XGBoost 14%, CAT 55%
    (0.32, 0.14, 0.54),  # LGB 32%, XGBoost 14%, CAT 54%
    (0.33, 0.14, 0.53),  # LGB 33%, XGBoost 14%, CAT 53%
    (0.30, 0.13, 0.57),  # LGB 30%, XGBoost 13%, CAT 57%
    (0.31, 0.13, 0.56),  # LGB 31%, XGBoost 13%, CAT 56%
    (0.32, 0.13, 0.55),  # LGB 32%, XGBoost 13%, CAT 55%
    (0.33, 0.13, 0.54),  # LGB 33%, XGBoost 13%, CAT 54%
    (0.30, 0.12, 0.58),  # LGB 30%, XGBoost 12%, CAT 58%
    (0.31, 0.12, 0.57),  # LGB 31%, XGBoost 12%, CAT 57%
    (0.32, 0.12, 0.56),  # LGB 32%, XGBoost 12%, CAT 56%
    (0.33, 0.12, 0.55),  # LGB 33%, XGBoost 12%, CAT 55%
]
best_auc = 0
best_ratio = None
best_ensemble_valid_proba = None

# 최적의 비율 찾기
for lgb_ratio, xgb_ratio, cat_ratio in ratios:
    ensemble_valid_proba = (lgb_ratio * lgb_val_pred_proba) + (xgb_ratio * xgb_val_pred_proba) + (cat_ratio * cat_val_pred_proba)
    auc = roc_auc_score(y_val, ensemble_valid_proba)
    print(f"LGB: {lgb_ratio}, XGB: {xgb_ratio}, CAT: {cat_ratio}, AUC: {auc:.6f}")
    
    if auc > best_auc:
        best_auc = auc
        best_ratio = (lgb_ratio, xgb_ratio, cat_ratio)
        best_ensemble_valid_proba = ensemble_valid_proba

# 최적 비율 출력
print(f"Best Ratio - LGB: {best_ratio[0]}, XGB: {best_ratio[1]}, CAT: {best_ratio[2]}, Best AUC: {best_auc:.10f}")

# 최적 비율을 사용하여 테스트 데이터 예측 계산
ensemble_test_proba = (best_ratio[0] * lgb_test_pred_proba) + (best_ratio[1] * xgb_test_pred_proba) + (best_ratio[2] * cat_test_pred_proba)

# 예측 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = ensemble_test_proba
sample_submission.to_csv('./02_27_XG_LGB_CAT_최적비율_11.csv', index=False)

LGB: 0.3, XGB: 0.15, CAT: 0.55, AUC: 0.737169
LGB: 0.31, XGB: 0.15, CAT: 0.54, AUC: 0.737170
LGB: 0.32, XGB: 0.15, CAT: 0.53, AUC: 0.737173
LGB: 0.33, XGB: 0.15, CAT: 0.52, AUC: 0.737174
LGB: 0.3, XGB: 0.16, CAT: 0.54, AUC: 0.737170
LGB: 0.31, XGB: 0.16, CAT: 0.53, AUC: 0.737174
LGB: 0.32, XGB: 0.16, CAT: 0.52, AUC: 0.737176
LGB: 0.33, XGB: 0.16, CAT: 0.51, AUC: 0.737177
LGB: 0.3, XGB: 0.17, CAT: 0.53, AUC: 0.737174
LGB: 0.31, XGB: 0.17, CAT: 0.52, AUC: 0.737176
LGB: 0.32, XGB: 0.17, CAT: 0.51, AUC: 0.737176
LGB: 0.33, XGB: 0.17, CAT: 0.5, AUC: 0.737176
LGB: 0.3, XGB: 0.18, CAT: 0.52, AUC: 0.737176
LGB: 0.31, XGB: 0.18, CAT: 0.51, AUC: 0.737178
LGB: 0.32, XGB: 0.18, CAT: 0.5, AUC: 0.737178
LGB: 0.33, XGB: 0.18, CAT: 0.49, AUC: 0.737177
LGB: 0.29, XGB: 0.15, CAT: 0.56, AUC: 0.737167
LGB: 0.3, XGB: 0.14, CAT: 0.56, AUC: 0.737165
LGB: 0.31, XGB: 0.14, CAT: 0.55, AUC: 0.737170
LGB: 0.32, XGB: 0.14, CAT: 0.54, AUC: 0.737172
LGB: 0.33, XGB: 0.14, CAT: 0.53, AUC: 0.737173
LGB: 0.3, XGB: 0.13,

In [79]:
ratios = [
    (0.29, 0.20, 0.51),  # LGB 29%, XGB 20%, CAT 51%
    (0.30, 0.19, 0.51),  # LGB 30%, XGB 19%, CAT 51%
    (0.31, 0.18, 0.51),  # LGB 31%, XGB 18%, CAT 51%
    (0.32, 0.17, 0.51),  # LGB 32%, XGBoost 17%, CAT 51%
    (0.33, 0.16, 0.51),  # LGB 33%, XGBoost 16%, CAT 51%
    (0.34, 0.15, 0.51),  # LGB 34%, XGBoost 15%, CAT 51%
    (0.35, 0.14, 0.51),  # LGB 35%, XGBoost 14%, CAT 51%
    (0.36, 0.13, 0.51),  # LGB 36%, XGBoost 13%, CAT 51%
    (0.37, 0.12, 0.51),  # LGB 37%, XGBoost 12%, CAT 51%
    (0.38, 0.11, 0.51),  # LGB 38%, XGBoost 11%, CAT 51%
]
best_auc = 0
best_ratio = None
best_ensemble_valid_proba = None

# 최적의 비율 찾기
for lgb_ratio, xgb_ratio, cat_ratio in ratios:
    ensemble_valid_proba = (lgb_ratio * lgb_val_pred_proba) + (xgb_ratio * xgb_val_pred_proba) + (cat_ratio * cat_val_pred_proba)
    auc = roc_auc_score(y_val, ensemble_valid_proba)
    print(f"LGB: {lgb_ratio}, XGB: {xgb_ratio}, CAT: {cat_ratio}, AUC: {auc:.6f}")
    
    if auc > best_auc:
        best_auc = auc
        best_ratio = (lgb_ratio, xgb_ratio, cat_ratio)
        best_ensemble_valid_proba = ensemble_valid_proba

# 최적 비율 출력
print(f"Best Ratio - LGB: {best_ratio[0]}, XGB: {best_ratio[1]}, CAT: {best_ratio[2]}, Best AUC: {best_auc:.10f}")

# 최적 비율을 사용하여 테스트 데이터 예측 계산
ensemble_test_proba = (best_ratio[0] * lgb_test_pred_proba) + (best_ratio[1] * xgb_test_pred_proba) + (best_ratio[2] * cat_test_pred_proba)

# 예측 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = ensemble_test_proba
sample_submission.to_csv('./02_27_XG_LGB_CAT_최적비율_12.csv', index=False)

LGB: 0.29, XGB: 0.2, CAT: 0.51, AUC: 0.737173
LGB: 0.3, XGB: 0.19, CAT: 0.51, AUC: 0.737174
LGB: 0.31, XGB: 0.18, CAT: 0.51, AUC: 0.737178
LGB: 0.32, XGB: 0.17, CAT: 0.51, AUC: 0.737176
LGB: 0.33, XGB: 0.16, CAT: 0.51, AUC: 0.737177
LGB: 0.34, XGB: 0.15, CAT: 0.51, AUC: 0.737176
LGB: 0.35, XGB: 0.14, CAT: 0.51, AUC: 0.737175
LGB: 0.36, XGB: 0.13, CAT: 0.51, AUC: 0.737173
LGB: 0.37, XGB: 0.12, CAT: 0.51, AUC: 0.737171
LGB: 0.38, XGB: 0.11, CAT: 0.51, AUC: 0.737172
Best Ratio - LGB: 0.31, XGB: 0.18, CAT: 0.51, Best AUC: 0.7371783875


In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

# Stacking을 위한 데이터 준비
kf = KFold(n_splits=10, shuffle=True, random_state=42)
meta_train = np.zeros((X.shape[0], 3))
meta_test = np.zeros((test.shape[0], 3))

for train_idx, val_idx in kf.split(X):
    X_train, X_valid = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[val_idx]

    # 개별 모델 학습
    lgb_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)
    cat_model.fit(X_train, y_train)

    # 개별 모델 예측 (Validation 데이터)
    meta_train[val_idx, 0] = lgb_model.predict_proba(X_valid)[:, 1]
    meta_train[val_idx, 1] = xgb_model.predict_proba(X_valid)[:, 1]
    meta_train[val_idx, 2] = cat_model.predict_proba(X_valid)[:, 1]

    # 개별 모델 예측 (Test 데이터)
    meta_test[:, 0] += lgb_model.predict_proba(test)[:, 1] / kf.n_splits
    meta_test[:, 1] += xgb_model.predict_proba(test)[:, 1] / kf.n_splits
    meta_test[:, 2] += cat_model.predict_proba(test)[:, 1] / kf.n_splits

# Meta-model 학습
meta_model = LogisticRegression()
meta_model.fit(meta_train, y)

# Stacking 모델 예측
final_preds = meta_model.predict_proba(meta_test)[:, 1]

C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:43:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:44:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [07:44:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\xgbo

In [110]:
# Meta-model 학습
meta_model = LogisticRegression()
meta_model.fit(meta_train, y)

# Stacking 모델 예측
final_preds = meta_model.predict_proba(meta_test)[:, 1]

from sklearn.metrics import roc_auc_score

# Meta-model이 훈련된 데이터에서 예측 수행
meta_train_preds = meta_model.predict_proba(meta_train)[:, 1]

# ROC-AUC 계산
stacking_auc = roc_auc_score(y, meta_train_preds)
print(f"Stacking Ensemble ROC-AUC: {stacking_auc:.6f}")

import pandas as pd

# 예측 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')  # 샘플 제출 파일 읽기
sample_submission['probability'] = final_preds  # final_preds를 probability 열에 저장

# 예측 결과를 새로운 CSV 파일로 저장
sample_submission.to_csv('./02_27_XG_LGB_CAT_Stacking.csv', index=False)
print("파일이 성공적으로 저장되었습니다.")

Stacking Ensemble ROC-AUC: 0.739881
파일이 성공적으로 저장되었습니다.


In [112]:
from sklearn.linear_model import RidgeClassifier

meta_model = RidgeClassifier(alpha=0.5)  # alpha 값으로 규제 강도 조절 가능
meta_model.fit(meta_train, y)
final_preds = meta_model.decision_function(meta_test)

# 예측 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')  # 샘플 제출 파일 읽기
sample_submission['probability'] = final_preds  # final_preds를 probability 열에 저장

# 예측 결과를 새로운 CSV 파일로 저장
sample_submission.to_csv('./02_27_XG_LGB_CAT_Ridge.csv', index=False)
print("파일이 성공적으로 저장되었습니다.")

from sklearn.metrics import roc_auc_score
import numpy as np

# decision_function으로 얻은 값을 확률로 변환 (Sigmoid 적용)
probabilities_train = 1 / (1 + np.exp(-meta_model.decision_function(meta_train)))

# ROC-AUC 계산
stacking_auc = roc_auc_score(y, probabilities_train)
print(f"Stacking Ensemble ROC-AUC: {stacking_auc:.6f}")

# test 데이터에 대한 예측
final_preds = 1 / (1 + np.exp(-meta_model.decision_function(meta_test)))  # 테스트 데이터에 대한 예측

파일이 성공적으로 저장되었습니다.
Stacking Ensemble ROC-AUC: 0.740026
